In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  # 設定 driver 的行為
from selenium.webdriver.support.ui import Select  # 選擇＂下拉式選單＂
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # 載入常見錯誤
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  # 更改載入策略
from selenium.webdriver.support.ui import WebDriverWait  # 等待機制
from selenium.webdriver.support import expected_conditions as EC  # 預期事件
from selenium.webdriver.common.by import By  # 找尋元素的方法
import pandas as pd  # 載入 pandas
import pandas.io.formats.excel  # 輸出自定義格式 Excel
import requests
import requests.packages.urllib3

requests.packages.urllib3.disable_warnings()  # 關閉錯誤警告
from urllib.request import HTTPError  # 載入 HTTPError
from bs4 import BeautifulSoup
import time  # 強制等待

In [3]:
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
my_capabilities = DesiredCapabilities.CHROME
my_capabilities[
    "pageLoadStrategy"] = "none"  # 當 html下載完成之後，不等待解析完成，selenium會直接返回

In [10]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)

In [ ]:
# 代入值
org='宜蘭縣公共圖書館'
org_url='https://webpac.ilccb.gov.tw/advanceSearch'
ISBN='9789573317241'

# 進入＂進階搜尋首頁＂
search_url = url + ISBN
driver.get(org_url)

# 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
search_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, 'searchField')))
select = Select(search_field)
time.sleep(0.5)
select.select_by_value('ISBN')

# 定位＂搜尋欄＂，輸入 ISBN
search_input = driver.find_element_by_name('searchInput')
search_input.send_keys(ISBN)
search_input.send_keys(Keys.ENTER)

# 計算＂搜尋結果的數量＂
book_list = wait.until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "div.booklist")))
results = len(book_list)
print(f"搜尋結果數量有：{results}筆", end="    ")

time.sleep(2)  # 頁面加載，用強制等待

# 依照搜尋結果的數量分成：一筆和多筆兩種情形
if results < 2:
    if WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.bookplace_list"))):
        click_more_btn(driver)
        return pd.read_html(driver.page_source, encoding="utf-8")[-1]
else:
    soup = BeautifulSoup(driver.page_source,
                         "html.parser")  # 用以當搜尋結果多筆時，找尋不同＂詳細書目＂的網址

    tgt = []
    url = url.split("search?")[0]  # 取得＂搜尋主頁＂頁面
    for li in soup.find_all("div", "booklist"):
        href = url + li.find("h2").find("a")["href"]

        driver.get(href)
        if wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.bookplace_list"))):
            click_more_btn(driver)
        tgt.append(
            pd.read_html(driver.page_source, encoding="utf-8")[-1])
    return pd.concat(tgt, axis=0, ignore_index=True)